# We read the data

In [39]:
import pandas as pd
data_2022 = pd.read_csv('meteo22.csv', sep=';')
data_2023 = pd.read_csv('meteo23.csv', sep=';')
data_2024 = pd.read_csv('meteo24.csv', sep=';')

data_meteo = pd.concat([data_2022, data_2023, data_2024], ignore_index=True)
data_meteo

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,V01,D02,...,D27,V27,D28,V28,D29,V29,D30,V30,D31,V31
0,28,79,102,81,28079102_81_98,2022,1,0.90,V,0.77,...,1.68,V,1.61,V,1.52,V,1.13,V,1.24,V
1,28,79,102,81,28079102_81_98,2022,2,4.13,V,1.04,...,1.20,V,1.09,V,0.00,N,0.00,N,0.00,N
2,28,79,102,81,28079102_81_98,2022,3,1.29,V,2.27,...,1.66,V,2.09,V,1.45,V,1.44,V,2.04,V
3,28,79,102,81,28079102_81_98,2022,4,2.68,V,2.76,...,1.35,V,1.35,V,2.07,V,1.18,V,0.00,N
4,28,79,102,81,28079102_81_98,2022,5,1.68,V,1.41,...,2.56,V,1.85,V,1.94,V,3.69,V,1.64,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,28,79,59,83,28079059_83_98,2024,3,9.90,V,9.40,...,6.80,V,8.10,V,7.80,V,0.00,N,0.00,N
2333,28,79,59,86,28079059_86_98,2024,3,71.00,V,73.00,...,55.00,V,58.00,V,66.00,V,0.00,N,0.00,N
2334,28,79,59,87,28079059_87_98,2024,3,954.00,V,952.00,...,938.00,V,942.00,V,939.00,V,0.00,N,0.00,N
2335,28,79,59,88,28079059_88_98,2024,3,133.00,V,137.00,...,184.00,V,201.00,V,193.00,V,0.00,N,0.00,N


In [40]:
magnitudes_dict = {
    80: 'ULTRAVIOLET RADIATION (Mw/m2)',
    81: 'WIND SPEED (m/s)',
    82: 'WIND DIRECTION (°)',
    83: 'TEMPERATURE (ºC)',
    86: 'RELATIVE HUMIDITY (%)',
    87: 'BARIOMETRIC PRESSURE (mb)',
    88: 'SOLAR RADIATION (W/m2)',
    89: 'PRECIPITATION (l/m2)'
}

# Create an empty DataFrame
df = pd.DataFrame(columns=['YEAR', 'MONTH', 'DAY', 'VALID'])

# Add columns for each magnitude from the dictionary
for magnitude_code, magnitude_name in magnitudes_dict.items():
    df[magnitude_name.upper()] = None

# Show the empty DataFrame
df
data_list = []
# Feed with the original dataframe
for index, row in data_meteo.iterrows():
    magnitude = magnitudes_dict[row['MAGNITUD']]
    year = row['ANO']
    month = row['MES']
    for day in range(1, 32):
        value_col = f'D{day:02}'
        validity_col = f'V{day:02}'
        value = row[value_col]
        validity = row[validity_col]
        data_list.append({
            'YEAR': year,
            'MONTH': month,
            'DAY': day,
            magnitude: value if validity == 'V' else None  # Assign None if validity is other than 'V'
        })

df = pd.DataFrame(data_list)

df = df.groupby(['YEAR', 'MONTH', 'DAY']).mean().reset_index()

# # There are some invalid dates
valid_dates  = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')
df = df.dropna()
df.drop(['YEAR', 'MONTH', 'DAY'], axis=1, inplace=True)
df.insert(0, 'Date', valid_dates.dt.strftime('%d/%m/%Y'))

df.to_csv('../meteoData.csv', index=False, sep=';')

df

,Date,WIND SPEED (m/s),WIND DIRECTION (°),TEMPERATURE (ºC),RELATIVE HUMIDITY (%),BARIOMETRIC PRESSURE (mb),SOLAR RADIATION (W/m2),PRECIPITATION (l/m2)
0,01/01/2022,0.487778,56.777778,9.686364,62.285714,953.857143,101.000000,0.000000
1,02/01/2022,0.475556,31.555556,8.945455,67.714286,954.428571,90.142857,0.000000
2,03/01/2022,0.552222,41.222222,8.981818,76.238095,951.714286,94.714286,0.000000
3,04/01/2022,1.390000,96.222222,7.150000,83.619048,943.142857,47.000000,4.655556
4,05/01/2022,1.643333,148.333333,6.286364,72.380952,944.571429,82.000000,3.766667
...,...,...,...,...,...,...,...,...
830,25/03/2024,3.433000,210.000000,9.268000,84.391304,938.875000,98.125000,0.430000
831,26/03/2024,2.465000,192.700000,8.660000,62.260870,935.375000,147.125000,0.030000
832,27/03/2024,1.899000,8.700000,7.508333,47.181818,943.500000,177.500000,0.000000
833,28/03/2024,1.471000,6.500000,8.658333,49.681818,946.875000,199.375000,0.000000


### We add the day of the week it is (Monday to Sunday) and if it was a public holiday in Madrid.

In [41]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Day Of Week'] = df['Date'].dt.day_name()

df

,Date,WIND SPEED (m/s),WIND DIRECTION (°),TEMPERATURE (ºC),RELATIVE HUMIDITY (%),BARIOMETRIC PRESSURE (mb),SOLAR RADIATION (W/m2),PRECIPITATION (l/m2),Day Of Week
0,2022-01-01,0.487778,56.777778,9.686364,62.285714,953.857143,101.000000,0.000000,Saturday
1,2022-01-02,0.475556,31.555556,8.945455,67.714286,954.428571,90.142857,0.000000,Sunday
2,2022-01-03,0.552222,41.222222,8.981818,76.238095,951.714286,94.714286,0.000000,Monday
3,2022-01-04,1.390000,96.222222,7.150000,83.619048,943.142857,47.000000,4.655556,Tuesday
4,2022-01-05,1.643333,148.333333,6.286364,72.380952,944.571429,82.000000,3.766667,Wednesday
...,...,...,...,...,...,...,...,...,...
830,2024-03-25,3.433000,210.000000,9.268000,84.391304,938.875000,98.125000,0.430000,Monday
831,2024-03-26,2.465000,192.700000,8.660000,62.260870,935.375000,147.125000,0.030000,Tuesday
832,2024-03-27,1.899000,8.700000,7.508333,47.181818,943.500000,177.500000,0.000000,Wednesday
833,2024-03-28,1.471000,6.500000,8.658333,49.681818,946.875000,199.375000,0.000000,Thursday


### Holidays

 https://www.elperiodico.com/es/economia/20211027/calendario-laboral-madrid-festivos-2022-12378519
 https://www.elperiodico.com/es/economia/20230213/calendario-laboral-madrid-festivos-2023-82902855
 https://www.comunidad.madrid/noticias/2023/09/27/comunidad-madrid-aprueba-calendario-laboral-2024-12-festivos-dos-fijen-ayuntamientos

#### Festivos en la Comunidad de Madrid para el 2022:

1 de enero: Año Nuevo

6 de enero: Reyes

14 de abril: Jueves Santo

15 de abril: Viernes Santo

16 de mayo: Traslado del día de San Isidro (fiesta local en Madrid ciudad)

2 de mayo: Fiesta de la Comunidad de Madrid

25 de julio: Santiago Apóstol

15 de agosto: La Asunción

12 de octubre: Día de la Hispanidad

1 de noviembre: Todos los Santos

9 de noviembre: La Almudena (fiesta local en Madrid ciudad)

6 de diciembre: La Constitución

8 de diciembre: La Inmaculada Concepción

26 de diciembre: Traslado del día de Navidad

#### Festivos en la Comunidad de Madrid para el 2023:

6 de enero: Reyes

20 de marzo: Traslado San José

6 de abril: Jueves Santo

7 de abril: Viernes Santo

1 de mayo: Día del Trabajo

15 de mayo: San Isidro

15 de agosto: Asunción de la Virgen

12 de octubre: Día de la Hispanidad

1 de noviembre: Todos los Santos

9 de noviembre: La Almudena

6 de diciembre: Día de la Constitución Española

8 de diciembre: Inmaculada Concepción

25 de diciembre: Navidad

#### Festivos en la Comunidad de Madrid para el 2024:

1 de enero: Año Nuevo

6 de enero: Epifanía del Señor

28 de marzo: Jueves Santo

29 de marzo: Viernes Santo

1 de mayo: Fiesta del Trabajador

2 de mayo: Fiesta de la Comunidad de Madrid

25 de julio: Santiago Apóstol

15 de agosto: Asunción de la Virgen

12 de octubre: Fiesta Nacional de España

1 de noviembre: Todos los Santos

6 de diciembre: Día de la Constitución Española

25 de diciembre: Natividad del Señor

In [42]:
festivos_madrid = [
    '01/01/2022',  # Año Nuevo
    '06/01/2022',  # Reyes
    '14/04/2022',  # Jueves Santo
    '15/04/2022',  # Viernes Santo
    '16/05/2022',  # Traslado San Isidro (fiesta local en Madrid ciudad)
    '02/05/2022',  # Fiesta de la Comunidad de Madrid
    '25/07/2022',  # Santiago Apóstol
    '15/08/2022',  # La Asunción
    '12/10/2022',  # Día de la Hispanidad
    '01/11/2022',  # Todos los Santos
    '09/11/2022',  # La Almudena (fiesta local en Madrid ciudad)
    '06/12/2022',  # La Constitución
    '08/12/2022',  # La Inmaculada Concepción
    '26/12/2022',  # Traslado Navidad
    
    '06/01/2023',  # Reyes
    '20/03/2023',  # Traslado San José
    '06/04/2023',  # Jueves Santo
    '07/04/2023',  # Viernes Santo
    '01/05/2023',  # Día del Trabajo
    '15/05/2023',  # San Isidro
    '15/08/2023',  # Asunción de la Virgen
    '12/10/2023',  # Día de la Hispanidad
    '01/11/2023',  # Todos los Santos
    '09/11/2023',  # La Almudena
    '06/12/2023',  # Día de la Constitución Española
    '08/12/2023',  # Inmaculada Concepción
    '25/12/2023',  # Navidad
    
    '01/01/2024',  # Año Nuevo
    '06/01/2024',  # Epifanía del Señor
    '28/03/2024',  # Jueves Santo
    '29/03/2024',  # Viernes Santo
    '01/05/2024',  # Fiesta del Trabajador
    '02/05/2024',  # Fiesta de la Comunidad de Madrid
    '25/07/2024',  # Santiago Apóstol
    '15/08/2024',  # Asunción de la Virgen
    '12/10/2024',  # Fiesta Nacional de España
    '01/11/2024',  # Todos los Santos
    '06/12/2024',  # Día de la Constitución Española
    '25/12/2024'   # Natividad del Señor
]


# Add 'Holiday' to DataFrame
df['Holiday'] = df['Date'].isin(pd.to_datetime(festivos_madrid, format='%d/%m/%Y'))
# Add the 'Tomorrow Holiday' column to the original DataFrame
df['Tomorrow Holiday'] = df['Date'] + pd.DateOffset(days=1)  # Get the next date
df['Tomorrow Holiday'] = df['Tomorrow Holiday'].isin(pd.to_datetime(festivos_madrid, format='%d/%m/%Y'))  # Check if the next date is a holiday

df = df.reindex(columns=['Date', 'Day Of Week', 'Holiday','Tomorrow Holiday', 'WIND SPEED (m/s)', 'WIND DIRECTION (°)', 'TEMPERATURE (ºC)', 'RELATIVE HUMIDITY (%)', 
                         'BARIOMETRIC PRESSURE (mb)', 'SOLAR RADIATION (W/m2)', 'PRECIPITATION (l/m2)'])

df.to_csv('../allCovariableData.csv', index=False, sep=';')
df

,Date,Day Of Week,Holiday,Tomorrow Holiday,WIND SPEED (m/s),WIND DIRECTION (°),TEMPERATURE (ºC),RELATIVE HUMIDITY (%),BARIOMETRIC PRESSURE (mb),SOLAR RADIATION (W/m2),PRECIPITATION (l/m2)
0,2022-01-01,Saturday,True,False,0.487778,56.777778,9.686364,62.285714,953.857143,101.000000,0.000000
1,2022-01-02,Sunday,False,False,0.475556,31.555556,8.945455,67.714286,954.428571,90.142857,0.000000
2,2022-01-03,Monday,False,False,0.552222,41.222222,8.981818,76.238095,951.714286,94.714286,0.000000
3,2022-01-04,Tuesday,False,False,1.390000,96.222222,7.150000,83.619048,943.142857,47.000000,4.655556
4,2022-01-05,Wednesday,False,True,1.643333,148.333333,6.286364,72.380952,944.571429,82.000000,3.766667
...,...,...,...,...,...,...,...,...,...,...,...
830,2024-03-25,Monday,False,False,3.433000,210.000000,9.268000,84.391304,938.875000,98.125000,0.430000
831,2024-03-26,Tuesday,False,False,2.465000,192.700000,8.660000,62.260870,935.375000,147.125000,0.030000
832,2024-03-27,Wednesday,False,True,1.899000,8.700000,7.508333,47.181818,943.500000,177.500000,0.000000
833,2024-03-28,Thursday,True,True,1.471000,6.500000,8.658333,49.681818,946.875000,199.375000,0.000000


### We draw out the enriched demand data 

In [43]:
demandData = pd.read_csv('../demandData.csv', sep=';')

demandData

# Same type variable 
demandData['Date'] = pd.to_datetime(demandData['Date'], format='%d/%m/%Y')
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

demandDataComplete = pd.merge(demandData, df, on='Date', how='left')

demandDataComplete.to_csv('../demandDataComplete.csv', index=False, sep=';')
demandDataComplete

,codnode,Pallets,Year,Month,Day,Date,Day Of Week,Holiday,Tomorrow Holiday,WIND SPEED (m/s),WIND DIRECTION (°),TEMPERATURE (ºC),RELATIVE HUMIDITY (%),BARIOMETRIC PRESSURE (mb),SOLAR RADIATION (W/m2),PRECIPITATION (l/m2)
0,167,19.00,2024,3,11,2024-03-11,Monday,False,False,1.740000,209.900000,5.5125,74.818182,938.285714,77.5,0.11
1,91,5.00,2024,3,11,2024-03-11,Monday,False,False,1.740000,209.900000,5.5125,74.818182,938.285714,77.5,0.11
2,228,8.00,2024,3,11,2024-03-11,Monday,False,False,1.740000,209.900000,5.5125,74.818182,938.285714,77.5,0.11
3,164,7.00,2024,3,11,2024-03-11,Monday,False,False,1.740000,209.900000,5.5125,74.818182,938.285714,77.5,0.11
4,184,20.00,2024,3,11,2024-03-11,Monday,False,False,1.740000,209.900000,5.5125,74.818182,938.285714,77.5,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37768,146,758.46,2022,6,30,2022-06-30,Thursday,False,False,1.536667,163.222222,23.9050,32.055556,939.000000,335.0,0.00
37769,23,196.81,2022,6,30,2022-06-30,Thursday,False,False,1.536667,163.222222,23.9050,32.055556,939.000000,335.0,0.00
37770,142,191.05,2022,6,30,2022-06-30,Thursday,False,False,1.536667,163.222222,23.9050,32.055556,939.000000,335.0,0.00
37771,70,98.52,2022,6,30,2022-06-30,Thursday,False,False,1.536667,163.222222,23.9050,32.055556,939.000000,335.0,0.00
